In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_purchases2020 = pd.read_csv("../data/raw/purchases_individuals_2020.csv")
df_purchases2019 = pd.read_csv("../data/raw/purchases_individuals_2019.csv")


df_purchases2020

In [ ]:
df_purchases2019

In [ ]:
df_purchases = pd.concat([df_purchases2019, df_purchases2020], ignore_index=True) #concatenating both datasets

df_purchases

In [ ]:
df_purchases_cleaned = df_purchases.dropna(axis=1, how='all') #removing all columns with all NaN values
df_purchases_cleaned = df_purchases_cleaned.dropna(axis=1, how='all') #removing all columns with all NaN values

df_purchases_cleaned

In [ ]:
df_purchases_cleaned["OBS_FLAG"].value_counts()

In [ ]:
df_purchases_cleaned_v2 = df_purchases_cleaned.drop(columns = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "OBS_FLAG"])

df_purchases_cleaned_v2

In [ ]:
df_purchases_cleaned_v3 = df_purchases_cleaned_v2.pivot(index=['indic_is', "ind_type", 'geo'], columns='TIME_PERIOD', values='OBS_VALUE').reset_index() #pivoting to add years on columns

df_purchases_cleaned_v3.columns.name = None # remove the name of the column index for better visualization

df_purchases_cleaned_v3

In [ ]:
df_purchases_cleaned_v3.rename(columns={"geo": "country", "indic_is": "last_online_purchase", "ind_type": "grouped_individuals"}, inplace=True)

df_purchases_cleaned_v3

In [ ]:
df_purchases_cleaned_v3.columns

In [ ]:
df_purchases_cleaned_v3['last_online_purchase'] = df_purchases_cleaned_v3['last_online_purchase'].str.replace("Last online purchase: ", "", regex=False)

df_purchases_cleaned_v3

In [ ]:
df_purchases_cleaned_v3["last_online_purchase"].value_counts()

In [ ]:
df_purchases_cleaned_v3["grouped_individuals"].value_counts()

In [ ]:
# i am going to filter by "last_online_purchase = in the 12 months" and "grouped_individuals = All individuals" to have a general view

df_purchases_global = df_purchases_cleaned_v3[df_purchases_cleaned_v3['last_online_purchase'] == "in the 12 months"]
df_purchases_global = df_purchases_global[df_purchases_global['grouped_individuals'] == "All individuals"].reset_index()
df_purchases_global = df_purchases_global.drop(columns = "index") # remove the column index for better visualization

#also removing the values "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)" or "European Union - 27 countries (from 2020)"
df_purchases_global = df_purchases_global[~df_purchases_global['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]

df_purchases_global

In [ ]:
df_cleaned = df_purchases_global.drop(columns=['last_online_purchase', 'grouped_individuals']) # dropping "last_online_purchase" and "grouped_individuals" columns	
df_cleaned

In [ ]:
df_cleaned.set_index('country', inplace=True) # set country as index

df_transposed = df_cleaned.T # transposing DF to have years as rows and country as columns

df_transposed.index = df_transposed.index.astype(int) # Convert the index (years) to integers for proper plotting /!\



In [ ]:
# Plot the data
plt.figure(figsize=(14, 8))
for country in df_transposed.columns:
    plt.plot(df_transposed.index, df_transposed[country], label=country)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('% of Individuals realizing Online Purchases')
plt.title('% of Individuals realizing Online Purchases by Country (2010-2024)')
plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')

# Add grid for better readability
plt.grid(True)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

# This is too messy and we cannot have conclusons

In [ ]:
# Lets try to simplify by grouping countries per regions
# Define regions - thanks AI :)
regions = {
    'Western Europe': ['Austria', 'Belgium', 'France', 'Germany', 'Netherlands', 'Switzerland', 'Luxembourg'],
    'Central Europe': ['Czechia', 'Hungary', 'Poland', 'Slovakia', 'Slovenia'],
    'Northern Europe': ['Denmark', 'Finland', 'Iceland', 'Ireland', 'Norway', 'Sweden', 'United Kingdom'],
    'Eastern Europe': ['Bulgaria', 'Croatia', 'Estonia', 'Latvia', 'Lithuania', 'Romania', 'Serbia', 'North Macedonia', 'Montenegro', 'Bosnia and Herzegovina', 'Albania', 'Kosovo*'],
    'Southern Europe': ['Greece', 'Italy', 'Malta', 'Portugal', 'Spain', 'Cyprus', 'Türkiye']
}

df_purchases_region = df_purchases_global.copy()

# Create a reverse mapping from country to region
country_to_region = {country: region for region, countries in regions.items() for country in countries}

# Add a 'region' column to the new DataFrame
df_purchases_region['region'] = df_purchases_region['country'].map(country_to_region)

# Drop unnecessary columns
df_cleaned = df_purchases_region.drop(columns=['last_online_purchase', 'grouped_individuals'])

# Group by region and calculate the mean for each year
df_grouped = df_cleaned.groupby('region').mean(numeric_only=True).T

# Convert the index (years) to integers for proper plotting
df_grouped.index = df_grouped.index.astype(int)

df_grouped

In [ ]:
# Plot the data
plt.figure(figsize=(14, 8))
for region in df_grouped.columns:
    plt.plot(df_grouped.index, df_grouped[region], marker='o', label=region)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('% of Individuals realizing Online Purchases')
plt.title('% of Individuals realizing Online Purchases by European Region (2010-2024)')
plt.legend(title='Region', loc='upper left')

# Add grid for better readability
plt.grid(True)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

# This one is looking good, maybe we can cross-check it with connection availability :)

In [ ]:
import seaborn as sns

# Transform DataFrame to a suitable format for a heatmap
heatmap_data = df_purchases_global.set_index('country').iloc[:, 3:]  # slice to get only year columns

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap='Blues', annot=False)
plt.title('Online Purchases Heatmap by Country and Year')
plt.show()


In [ ]:
df_purchases_global.columns = df_purchases_global.columns.astype(str)
df_2020 = df_purchases_global[['country', '2020']].dropna().sort_values('2020', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='2020', y='country', data=df_2020)
plt.xlabel('Percentage of Online Purchases in 2020')
plt.title('Online Purchases by Country in 2020')
plt.show()

In [ ]:
# now time to check how education level affects this

In [ ]:
df_purchases_cleaned_v3

In [ ]:
df_purchases_cleaned_v3["grouped_individuals"].value_counts()

In [ ]:
# i am going to filter by "last_online_purchase = in the 12 months" and "grouped_individuals = educations" to have the detail
educationlevel = ["Individuals aged 16-24 with high formal education", "Individuals aged 16-24 with low education", "Individuals aged 16-24 with medium formal education", "Individuals aged 25 to 54 with high formal education", "Individuals aged 25 to 54 with low formal education", "Individuals aged 25 to 54 with medium formal education", "Individuals aged 55 to 74 with high formal education", "Individuals aged 55 to 74 with low formal education", "Individuals aged 55 to 74 with medium formal education"]

df_purchases_education = df_purchases_cleaned_v3[df_purchases_cleaned_v3['last_online_purchase'] == "in the 12 months"]
df_purchases_education = df_purchases_education[df_purchases_education['grouped_individuals'].isin(educationlevel)].reset_index(drop=True)

#also removing the values "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)" or "European Union - 27 countries (from 2020)"
df_purchases_education = df_purchases_education[~df_purchases_education['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]

df_purchases_education = df_purchases_education.drop(columns="last_online_purchase") # drop last_online_purchase column

df_purchases_education

In [ ]:
df_purchases_education["grouped_individuals"].value_counts()

In [ ]:
# Lets try to group by education level low - medium - high
# Define education
education = {
    'Low': ['Individuals aged 16-24 with low education', 'Individuals aged 25 to 54 with low formal education', 'Individuals aged 55 to 74 with low formal education'],
    'Medium': ['Individuals aged 16-24 with medium formal education', 'Individuals aged 25 to 54 with medium formal education', 'Individuals aged 55 to 74 with medium formal education'],
    'High': ['Individuals aged 16-24 with high formal education', 'Individuals aged 25 to 54 with high formal education', 'Individuals aged 55 to 74 with high formal education'],
}

df_purchases_edugrouped = df_purchases_education.copy() # Reverse mapping from education description to level /!\

education_to_level = {education_desc: level for level, descriptions in education.items() for education_desc in descriptions}

# Add a new 'education_level' column using the mapping
df_purchases_edugrouped['education_level'] = df_purchases_edugrouped['grouped_individuals'].map(education_to_level)

df_cleaned2 = df_purchases_edugrouped.drop(columns='grouped_individuals') # Drop no needed columns

df_grouped2 = df_cleaned2.groupby('education_level').mean(numeric_only=True).T # Group by the 'education_level' and calculate the mean for each year

df_grouped2.index = df_grouped2.index.astype(int) # Convert the index (years) to integers for proper plotting

df_grouped2

In [ ]:
# Define the DataFrame (assuming df_grouped2 is already created as you described)
# Columns are 'High', 'Low', 'Medium' and index are the years

plt.figure(figsize=(14, 8))

# Iterate over each education level and plot its respective line on the graph
for education_level in df_grouped2.columns:
    plt.plot(df_grouped2.index, df_grouped2[education_level], marker='o', label=education_level)

# Add labels and a title
plt.xlabel('Year')
plt.ylabel('% of Individuals Realizing Online Purchases')
plt.title('% of Individuals Realizing Online Purchases by Education Level (2010-2024)')

# Add a legend to identify education levels in the plot
plt.legend(title='Education Level', loc='upper left')

# Add a grid to improve readability
plt.grid(True)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Optimize layout to prevent clipping
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# want to do the same grouping per ages, just in case we detect something interesting:

In [ ]:
# removing all row values that add have more info than just group age.
# list with the values to keep:
age_groups = ["Individuals, 16 to 24 years old", "Individuals, 25 to 34 years old", "Individuals, 35 to 44 years old", "Individuals, 45 to 54 years old", "Individuals, 55 to 64 years old", "Individuals, 65 to 74 years old"]

df_purchases_agegrouped = df_purchases_cleaned_v3[df_purchases_cleaned_v3['last_online_purchase'] == "in the 12 months"]
df_purchases_agegrouped = df_purchases_agegrouped[df_purchases_agegrouped['grouped_individuals'].isin(age_groups)].reset_index(drop=True)

df_purchases_agegrouped = df_purchases_agegrouped.drop(columns='last_online_purchase') # Drop no needed columns

#also removing the values "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)" or "European Union - 27 countries (from 2020)"
df_purchases_agegrouped = df_purchases_agegrouped[~df_purchases_agegrouped['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]


df_purchases_agegrouped

In [ ]:
# I am detecting that those NaN seem to be values = 0 in most of cases so going to fill those NaN with 0:
df_purchases_agegrouped.fillna(0, inplace=True)

In [ ]:
# going to group by grouped_individuals mean:
df_age_meanvalues = df_purchases_agegrouped.groupby('grouped_individuals').mean(numeric_only=True)

df_age_meanvalues

In [ ]:
df_age_meanvalues = df_age_meanvalues.T # TRANSPOSING!

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
for column in df_age_meanvalues.columns:
    plt.plot(df_age_meanvalues.index, df_age_meanvalues[column], marker='o', label=column)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Value')
plt.title('Line Graph of Values by Age Group (2010-2024)')
plt.legend(title='Age Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# Individuals living in cities
# Individuals living in rural areas 
# Individuals living in towns and suburbs
# we can use that to compare it with the available conectivity

In [ ]:
# list with the values to keep:
living_area = ["Individuals living in cities", "Individuals living in rural areas", "Individuals living in towns and suburbs"]

df_purchases_livinggroup = df_purchases_cleaned_v3[df_purchases_cleaned_v3['last_online_purchase'] == "in the 12 months"]
df_purchases_livinggroup = df_purchases_livinggroup[df_purchases_livinggroup['grouped_individuals'].isin(living_area)].reset_index(drop=True)

df_purchases_livinggroup = df_purchases_livinggroup.drop(columns='last_online_purchase') # Drop no needed columns

#also removing the values "Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)" or "European Union - 27 countries (from 2020)"
df_purchases_livinggroup = df_purchases_livinggroup[~df_purchases_livinggroup['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]


df_purchases_livinggroup

In [ ]:
# going to group by grouped_individuals mean:
df_living_meanvalues = df_purchases_livinggroup.groupby('grouped_individuals').mean(numeric_only=True)

df_living_meanvalues

In [ ]:
df_living_meanvalues = df_living_meanvalues.T # TRANSPOSING!

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
for column in df_living_meanvalues.columns:
    plt.plot(df_living_meanvalues.index, df_living_meanvalues[column], marker='o', label=column)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Value')
plt.title('Line Graph of Values by Living Area (2010-2024)')
plt.legend(title='Living Area Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# in case we want to have the main data source filtered by European regions:

df_per_region = df_purchases_cleaned_v3.copy()

# Create a reverse mapping from country to region
country_to_region = {country: region for region, countries in regions.items() for country in countries}

# Add a 'region' column to the new DataFrame
df_per_region['region'] = df_per_region['country'].map(country_to_region)


df_per_region

In [ ]:
df_per_region["grouped_individuals"].value_counts()

In [ ]:
#testing something
df_per_regionall = df_per_region[df_per_region['last_online_purchase'] == "in the 12 months"]
df_per_regionall = df_per_regionall.drop(columns = "last_online_purchase") # remove the extra columns
df_per_regionall = df_per_regionall[~df_per_regionall['country'].isin(["Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)", "European Union - 27 countries (from 2020)"])]
df_per_regionall = df_per_regionall[df_per_regionall['grouped_individuals'].isin(living_area)].reset_index(drop=True)

df_per_regionall